In [1]:
import os
import mlflow
from dotenv import load_dotenv
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_community.callbacks.mlflow_callback import MlflowCallbackHandler

In [2]:
import logging
import warnings

# Suppress MLflow autologging warnings
warnings.filterwarnings("ignore", module="mlflow")

# Target just the mlflow.langchain logger
logging.getLogger("mlflow.langchain._langchain_autolog").setLevel(logging.ERROR)

In [3]:
# if using sklearn models
mlflow.autolog()
# Set experiment name (this will create it if it doesn't exist)
mlflow.set_experiment("New Experiment Langgraph")
# If using OpenAI models, you can enable autologging for OpenAI
mlflow.openai.autolog()

2025/08/16 19:19:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [4]:
# Load variables from .env file
load_dotenv()

# Access variables
api_key_ = os.getenv("OPENAI_API_KEY")

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    max_tokens=1000,
    streaming=True,
    verbose=True,
    callbacks=[MlflowCallbackHandler()]
)

Run `python -m spacy download en_core_web_sm` to download en_core_web_sm model for text visualization.


In [5]:
class WeatherResponse(BaseModel):
    conditions: str = Field(
        description="The weather conditions in the specified city.",
        example="rainy"
    )

@tool
def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]

2025/08/16 19:19:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for langchain.
2025/08/16 19:19:10 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: Span for run_id 2d55c6b0-a289-4463-bb1b-39f7b27a892e not found.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025/08/16 19:19:11 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: Span for run_id b931680b-b3b6-45a3-bd0e-a0fcc0e7efe7 not found.
2025/08/16 19:19:11 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: Span for run_id dc176f69-2a36-4429-9620-7bdbc6f3ea77 not found.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025/08/16 19:19:12 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: Span for run_id 3c023976-85b7-4170-a85a-61b380e1ad12 not found.
INFO:httpx:HTTP Request: POST

WeatherResponse(conditions='sunny')